In [50]:
import json
import prettytable
from neo4j import GraphDatabase
from dotenv import load_dotenv
from gen_query import handle_gen_query
import util
from get_entities import handle_get_entities

load_dotenv()


False

In [51]:
def gen_query_ontology(text):
    table = prettytable.PrettyTable(["Step", "Result"])
    table.max_width = 80
    table.add_row(["Input", text])

    concepts = util.get_concepts(text)

    match_relation = util.get_match_relation(concepts.keys())

    evidences = handle_get_entities(text)

    table.add_rows([["Match alias", concepts], ["Find individuals",
                                                evidences]])

    targets = list(set(concepts.keys()).difference(set(evidences.keys())))
    table.add_row(["Target concepts", targets])

    query = handle_gen_query(concepts.keys(), evidences, match_relation)
    table.add_row(["Query", query])

    print(table)
    # print(query)
    return query

In [53]:
# text = "nhà hoặc căn hộ giá khoảng 2 tỷ thì mua ở quận nào"
text = "nhà ở quận 1 thường có giá khoảng bao nhiêu"

cqlNodeQuery = gen_query_ontology(text)

+------------------+----------------------------------------------------------------------------------+
|       Step       |                                      Result                                      |
+------------------+----------------------------------------------------------------------------------+
|      Input       |                   nhà ở quận 1 thường có giá khoảng bao nhiêu                    |
|   Match alias    | {'real_estate_type': 'nhà', 'price': 'giá khoảng bao nhiêu', 'district': 'quận'} |
| Find individuals |                               {'district': ['1']}                                |
| Target concepts  |                          ['price', 'real_estate_type']                           |
|      Query       |                                                                                  |
|                  |                   MATCH (real_estate_type:House)-->(price:Price),                |
|                  |                  (real_estate_type:House)--

In [22]:
# Connect neo4j DB.
driver = GraphDatabase.driver('bolt://localhost:7687',
                              auth=('neo4j', 'password'))


In [23]:
def run_query(query):
    with driver.session(database="htdb") as session:
        results = session.run(query)

        table_results = prettytable.PrettyTable(results.keys())
        table_results.max_width = 60
        for r in results:
            table_results.add_row(r.values())

        return table_results
        # return results


In [24]:
result_data = run_query(cqlNodeQuery)

print(result_data)

+----------------------------------------------------+
|                      District                      |
+----------------------------------------------------+
| ['hoang mai', '5 tu liem', 'long bien', 'kien an'] |
+----------------------------------------------------+


In [26]:
with open('data/question_dict.json', 'r', encoding="utf-8") as fp:
    question_dict = json.load(fp)
    fp.close()

REAL_ESTATE_TYPE = question_dict["real_estate_type"]
PRICE = question_dict["price"]
AREA = question_dict["area"]
DISTRICT = question_dict["district"]
BED_ROOM = question_dict["bed_room"]
FLOOR = question_dict["floor"]
LEGAL = question_dict["legal"]
POSITION = question_dict["position"]
LOCATION = question_dict["location"]
POTENTIAL = question_dict["potential"]